In [ ]:
input_path = 'smbg_profile_data'
output_path = 'smbg_profile_data_formatted'

In [ ]:
import pandas as pd

In [ ]:
smbg = pd.read_csv(f'{input_path}/DCCTID{PID}.csv',
                   parse_dates=['Date']) \
            .rename(columns={'time': 'Time',
                             'ID': 'PID',
                             'probability': 'Probability',
                             'DCCT Flag': 'DCCT_Flag'})
print(smbg.shape[0])
smbg.head(3)

### Drop rows where the SMBG value is NAN

In [ ]:
smbg = smbg.loc[smbg['SMBG'].notnull()]
smbg.shape

### Drop duplicate rows

In [ ]:
smbg.drop_duplicates(subset=['PID', 'Date', 'Time'], inplace=True)
smbg.shape

### Adjust 'Date' and 'Time' when the time is > 24 (i.e., SMBG value is from the next day)

In [ ]:
smbg['Date'] = smbg.apply(lambda row: row['Date'] + pd.DateOffset(days=1) if row['Time'] >= 24 else row['Date'],
                          axis=1)
smbg['Time'] = smbg['Time'].apply(lambda t: t % 24)

### Add in Date_Time column

In [ ]:
smbg['Hours'] = smbg['Time'].astype(int)
smbg['Minutes'] = smbg['Time'].apply(lambda t: int((t % 1) * 60))
smbg['Date_Time'] = smbg.apply(lambda row: row['Date'] + pd.DateOffset(hours=row['Hours'], minutes=row['Minutes']),
                               axis=1)

### Add in Time_Index column

In [ ]:
smbg['Time_Index'] = ((smbg['Date_Time'].dt.hour * 60) + smbg['Date_Time'].dt.minute) / 5
smbg['Time_Index'] = smbg['Time_Index'].astype(int)

### Check that two or more SMBG values do not occur during the same 5-minute time interval.  If they do, average the SMBG values and remove all but one row of data

In [ ]:
num_smbg_val_per_dti = smbg.groupby(['Date', 'Time_Index']).size()
dti_gt1 = num_smbg_val_per_dti.loc[num_smbg_val_per_dti > 1].index
dti_gt1

In [ ]:
print(smbg.shape)
new_rows = []
for date, time_index in dti_gt1:
    mask = (smbg['Date'] == date) & (smbg['Time_Index'] == time_index)   
    new_rows.append(smbg.loc[mask].iloc[0])
    new_rows[-1]['SMBG'] = int(smbg.loc[mask]['SMBG'].mean())
    
    smbg = smbg.loc[~mask]
    print(smbg.shape)

In [ ]:
smbg = pd.concat([smbg, pd.DataFrame(new_rows)], axis=0) \
            .sort_values('Date_Time') \
            .reset_index(drop=True)
print(smbg.shape)
smbg.head(3)

### Check that 5 mg/dL/min change in BG values assumption is not violated

In [ ]:
smbg = smbg.sort_values('Date_Time').reset_index(drop=True)

In [ ]:
delta_gt5mgdLmin_violation = (smbg['SMBG'].diff(1) / (smbg['Date_Time'].diff(1).dt.total_seconds() / 60)).abs() > 10

In [ ]:
if PID not in [1120]:
    assert delta_gt5mgdLmin_violation.sum() == 0, 'Violation'

### Censor values outside CGM bounds

In [ ]:
smbg['SMBG_Uncensored'] = smbg.loc[:, ['SMBG']]

In [ ]:
smbg.loc[smbg['SMBG'] < 40, 'SMBG'] = 39
smbg.loc[smbg['SMBG'] > 400, 'SMBG'] = 401

### Check that there is data for each day between the start date and the end date

In [ ]:
min_date = smbg['Date'].min()
print(min_date.strftime('%Y-%m-%d'))
max_date = smbg['Date'].max()
print(max_date.strftime('%Y-%m-%d'))
n_unique_dates = len(smbg['Date'].unique())
print(n_unique_dates)

assert n_unique_dates - ((max_date - min_date).days + 1) == 0, 'Violation'

### Write out data

In [ ]:
smbg[['PID', 'Date', 'Date_Time', 'Time_Index', 'SMBG', 'SMBG_Uncensored']] \
    .to_csv(f'{output_path}/pid{PID}_smbg_profile_data.csv', index=False)